### 1. Load and preprocess data

In [1]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models, regularizers, callbacks

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import json
import keras
from utils.textpreprocessing import TextPreprocessing

In [2]:
model = models.load_model('models/sarcasm_2.keras')

In [ ]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 85, 10)              │          80,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ (None, 85, 20)              │           1,680 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_3 (Bidirectional)      │ (None, 20)                  │           2,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 24)                  │             504 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              25 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 254,069 (992.46 KB)

 Trainable params: 84,689 (330.82 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 169,380 (661.64 KB)

In [ ]:
utils_path = '/content/drive/MyDrive/Colab Notebooks/XAI/utils'

sys.path.insert(0, utils_path)

In [ ]:
text_preprocessing = TextPreprocessing()

In [5]:
with open("sarcasm.json", 'r') as f:
    datastore = json.load(f)

In [ ]:
vocab_size = 8000
embedding_dim = 10
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = ""
training_size = 23000

In [ ]:
text_preprocessing.set_datastore(datastore)
text_preprocessing.clean_datastore()
text_preprocessing.initize_tokenizer()
X_train_padded, X_test_padded, y_train, y_test = text_preprocessing.get_data()

/content/drive/MyDrive/Colab Notebooks/XAI/utils/textpreprocessing.py:37: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 37 of the file /content/drive/MyDrive/Colab Notebooks/XAI/utils/textpreprocessing.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(sentence)
/content/drive/MyDrive/Colab Notebooks/XAI/utils/textpreprocessing.py:37: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(sentence)


In [ ]:
y_pred = model.predict(X_test_padded)

116/116 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step


### 2. LIME

In [ ]:
!pip install lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=0004d989b6792ff7b7faa661f873ef6f8b6f754391ae41e3a6519dbca9d348ea
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


In [ ]:
import lime
from lime import lime_text

In [104]:
def predict_fn(x):
  text = text_preprocessing.create_sequences(x, 85, 'post', 'post')
  return model.predict(text)

In [105]:
explainer = lime_text.LimeTextExplainer(class_names=['not sarcasm', 'sarcasm'])

In [102]:
data = text_preprocessing.sentences
row = data[training_size]

# row = X_test_padded[0]

In [106]:
explanation = explainer.explain_instance(row, predict_fn, num_features=2)

TypeError: cannot use a string pattern on a bytes-like object

### 3. SHAP

In [ ]:
!pip install shap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.1/540.1 kB 29.4 MB/s eta 0:00:00


In [ ]:
!pip install nltk

In [ ]:
import shap
import tensorflow.python.keras.backend as K

In [ ]:
row = X_test_padded[0]
row = row.reshape(1, -1)

In [109]:
tokenizer = text_preprocessing.tokenizer

word_index = tokenizer.word_index

In [113]:
session = K.get_session()

explainer = shap.Explainer(model, word_index)

In [114]:
shap_values = explainer(row)

TypeError: 'numpy.float64' object cannot be interpreted as an integer